# Import all the libraries 

In [1]:
from pymongo import MongoClient
import re
from pymongo import ASCENDING, DESCENDING
import pandas as pd
from pandas.io.json import json_normalize
import requests



# Import the dataset Companies

In [2]:
dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


# Geocoding

Get the longitude and latitude though an address.

In [3]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates": [float(data["longt"]), float(data["latt"])]
    }

In [25]:
a = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))

In [4]:
all_offices = list(db.companies.find({},{"offices":1,"name":1,"category_code":1}))
all_offices

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'category_code': 'web',
  'offices': [{'description': '',
    'address1': '710 - 2nd Avenue',
    'address2': 'Suite 1100',
    'zip_code': '98104',
    'city': 'Seattle',
    'state_code': 'WA',
    'country_code': 'USA',
    'latitude': 47.603122,
    'longitude': -122.333253},
   {'description': '',
    'address1': '270 Lafayette Street',
    'address2': 'Suite 505',
    'zip_code': '10012',
    'city': 'New York',
    'state_code': 'NY',
    'country_code': 'USA',
    'latitude': 40.7237306,
    'longitude': -73.9964312}]},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'category_code': 'enterprise',
  'offices': [{'description': 'Headquarters',
    'address1': '4900 Hopyard Rd.',
    'address2': 'Suite 310',
    'zip_code': '94588',
    'city': 'Pleasanton',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.692934,
    'longitude': -121.904945}]},
 {'_id': ObjectId('

In [5]:
from pandas.io.json import json_normalize
all_offices = pd.DataFrame(json_normalize(all_offices))
all_offices.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"[{'description': '', 'address1': '710 - 2nd Av..."
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"[{'description': '', 'address1': 'Suite 200', ..."
4,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."


In [26]:
c = pd.DataFrame(json_normalize(a))
c.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


In [6]:
companydata = pd.DataFrame(all_offices)
companydata.head()

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"[{'description': '', 'address1': '710 - 2nd Av..."
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"[{'description': '', 'address1': 'Suite 200', ..."
4,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."


In [7]:
b = companydata.explode(column="offices")
b

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
1,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"{'description': 'Headquarters', 'address1': '4..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,"{'description': 'Headquarters', 'address1': '4..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"{'description': '', 'address1': 'Suite 200', '..."
...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,"{'description': '', 'address1': '', 'address2'..."
18797,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,"{'description': '', 'address1': '1900 Internat..."
18798,52cdef7f4bab8bd67529c6f7,AfterLogic,software,"{'description': 'Livingston', 'address1': 'P.O..."
18799,52cdef7f4bab8bd67529c6f1,Willdan Group,other,"{'description': '', 'address1': '2401 East Kat..."


In [8]:
a = companydata["offices"][0]
a

[{'description': '',
  'address1': '710 - 2nd Avenue',
  'address2': 'Suite 1100',
  'zip_code': '98104',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.603122,
  'longitude': -122.333253},
 {'description': '',
  'address1': '270 Lafayette Street',
  'address2': 'Suite 505',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'latitude': 40.7237306,
  'longitude': -73.9964312}]

In [9]:
def officeToGeoPoint(row):
    office = row.offices
    if type(office) == dict:
        if 'latitude' in office and 'longitude' in office:
            if(type(office["latitude"])) == float and type(office["longitude"]) == float:
                return ({
                    "type":"Point",
                    "coordinates":[office["longitude"],office["latitude"]]
                },"success")
            else:
                return(None,"Invalid lat lat and long")
        else:
            return (None,"No lat and long keys in office dict")
    return (None,"No office")

In [15]:
cleaned_offices = companydata.apply(officeToGeoPoint,axis=1, result_type="expand")
cleaned_offices.columns = ["office","clean_state"]

cleaned_offices

,office,clean_state
0,None,No office
1,None,No office
2,None,No office
3,None,No office
4,None,No office
...,...,...
18796,None,No office
18797,None,No office
18798,None,No office
18799,None,No office


In [11]:
company_processed = pd.concat([companydata,cleaned_offices], axis=1)

In [12]:
company_processed = company_processed[["name","category_code","office","clean_state"]]

In [13]:
company_processed

,name,category_code,office,clean_state
0,Wetpaint,web,None,No office
1,AdventNet,enterprise,None,No office
2,Zoho,software,None,No office
3,Omnidrive,network_hosting,None,No office
4,Postini,web,None,No office
...,...,...,...,...
18796,EnteGreat Solutions,software,None,No office
18797,EnteGreat,enterprise,None,No office
18798,AfterLogic,software,None,No office
18799,Willdan Group,other,None,No office


In [14]:
company_processed.clean_state.value_counts()

No office    18801
Name: clean_state, dtype: int64